In [1]:
! pip install -U spacy -q

In [2]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.8.7                         
Location         /usr/local/lib/python3.12/dist-packages/spacy
Platform         Linux-6.6.105+-x86_64-with-glibc2.35
Python version   3.12.12                       
Pipelines        en_core_web_sm (3.8.0)        



In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [13]:
import json
f = open('/content/Financial_Insignts_annotations.json')
TRAIN_DATA = json.load(f)

In [14]:
TRAIN_DATA

[['Apple Inc. (AAPL) announced a 15% increase in revenue for Q3 2025, totaling $95 billion.',
  {'entities': [[0, 9, 'ORG'],
    [11, 15, 'STOCK'],
    [33, 36, 'PERCENTAGE'],
    [63, 70, 'DATE'],
    [81, 91, 'MONEY']]}],
 ['CEO Tim Cook stated that sales of the iPhone 15 and MacBook Pro were the main contributors.',
  {'entities': [[97, 105, 'PEOPLE'],
    [139, 148, 'PRODUCT'],
    [153, 164, 'PRODUCT']]}],
 ['The company expects continued growth in Europe and Asia-Pacific regions over the next fiscal year.',
  {'entities': [[207, 213, 'CONTINENT'],
    [218, 230, 'CONTINENT'],
    [247, 262, 'TIME']]}]]

In [15]:
for text, annot in tqdm(TRAIN_DATA):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy")

100%|██████████| 3/3 [00:00<00:00, 798.31it/s]

Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity
Skipping entity


In [16]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency


✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [17]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     28.43   33.33   33.33   33.33    0.33
200     200          1.47    437.66  100.00  100.00  100.00    1.00
400     400          0.00      0.00  100.00  100.00  100.00    1.00
600     600          0.00      0.00  100.00  100.00  100.00    1.00
800     800          0.00      0.00  100.00  100.00  100.00    1.00
1000    1000          0.00      0.00  100.00  100.00  100.00    1.00
1200    1200          0.00      0.00  100.00  100.00  100.00    1.00
1400    1400          0.00      0.00  100.00  100.00  100.00    1.00
1600    1600          0.00      0

In [18]:
nlp_ner = spacy.load("/content/model-best")

In [19]:
doc = nlp_ner("Market analysts at MorganEast Research, based in Singapore, forecast that the Indian digital lending sector will surpass ₹1.8 lakh crore in transaction volume by 2027, with Aurora capturing approximately 6.4% market share.")